In [24]:

#  Setting display options 

import pandas as pd
   #
pd.set_option("display.width", 480)

#  Sets horizontal scroll for wide outputs
#
from IPython.display import display, HTML
display(HTML(""))

from tabulate import tabulate

print("--")


--


#  Setup stuff: Connectivity

In [25]:

from katana import remote
from katana.remote import import_data


my_client = remote.Client()

print(my_client)


In [26]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [27]:

#  DELETE ALL GRAPHS

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name=l_database.name).graphs_in_database():
      l_handle=my_client.get_database(name=l_database.name).get_graph_by_id(id=l_graph.graph_id)
      l_handle.delete()

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "      GRAPH Version: ", l_graph.version)

print("--")


--


In [ ]:

#  DELETE ALL DATABASES

for l_database in my_client.databases():
   if (l_database.name != "default"):
      my_client.get_database(name=l_database.name).delete_database()
      print("--")

for l_database in my_client.databases():
   print("DB ID: ", l_database.database_id, "     DB Name: ", l_database.name)


In [ ]:

#  CREATE DATABASE

my_database = my_client.create_database(name=DB_NAME)

print(my_database.database_id)


In [28]:

#  CREATE GRAPH

my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


<_Graph my_graph, 57WmShSmPH1jVLmTZRa5YrTGDUu8wmWLRAbfmCTSKzUn, 0>


In [ ]:

#  CONNECT TO GRAPH

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:
# import pandas as pd
# 
# print("--")


In [29]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   #  ], columns = ["airport_code", "airport_name", "LABEL"])
   ], columns = ["id", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


+----+------+----------------+---------+
|    | id   | airport_name   | LABEL   |
|----+------+----------------+---------|
|  0 | MKE  | Milwaukee      | Airport |
|  1 | ORD  | Chicago O-Hare | Airport |
|  2 | SJC  | San Jose       | Airport |
|  3 | DEN  | Denver         | Airport |
+----+------+----------------+---------+


In [30]:

#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

print(tabulate(df_flights, headers='keys', tablefmt='psql'))


+----+------------+----------+------------+------------+----------+
|    | START_ID   | END_ID   |   DISTANCE |   NUM_HOPS | TYPE     |
|----+------------+----------+------------+------------+----------|
|  0 | MKE        | ORD      |         66 |          1 | FLIES_TO |
|  1 | ORD        | MKE      |         66 |          1 | FLIES_TO |
|  2 | ORD        | DEN      |        886 |          1 | FLIES_TO |
|  3 | DEN        | ORD      |        886 |          1 | FLIES_TO |
|  4 | SJC        | DEN      |        948 |          1 | FLIES_TO |
|  5 | DEN        | SJC      |        948 |          1 | FLIES_TO |
|  6 | SJC        | ORD      |       1829 |          1 | FLIES_TO |
+----+------------+----------+------------+------------+----------+


In [31]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiports set of Nodes
      #  id_column             = "airport_code",
      id_column             = "id",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


          0/? [?op/s]

          0/? [?op/s]

--


In [32]:

#  Used to test heterogeneous graphs ..

l_query = """
   CREATE ( n: Restaurant { id: 'PAP' } )                         //  Notice single curly braces
   SET n.restaurant_name = 'Pappadeauxs Seafood Kitchen' 
   """
      #
display(l_result1 = my_graph.query(l_query))

l_query = """
   MATCH
      (n: Airport   ),
      (m: Restaurant)
   WHERE n.id               = 'SJC'
   AND   m.id               = 'PAP'
   CREATE (n) -[r: Attraction { DRIVABLE: 'Yes' }]-> (m)
   """
      #
display(l_result2 = my_graph.query(l_query))


          0/? [?op/s]

          0/? [?op/s]

#  Diagnostics

In [23]:

#  What's in the graph ..

l_result1 = my_graph.query("""
   MATCH ( n ) 
   RETURN n
   """)
print(tabulate(l_result1, headers='keys', tablefmt='psql'))

l_result2 = my_graph.query("""
   MATCH ( n ) - [ r ] -> ( m )
   RETURN r
   """)
# print(l_result2[0:30])
print(tabulate(l_result2, headers='keys', tablefmt='psql'))



          0/? [?op/s]

+----+-----------------+----------------+--------+-----------------------------+----------+-----------+------------------+
|    |   n.internal_id | n.labels       | n.id   | n.restaurant_name           | n.type   | n.LABEL   | n.airport_name   |
|----+-----------------+----------------+--------+-----------------------------+----------+-----------+------------------|
|  0 |               0 | ['Restaurant'] | PAP    | Pappadeauxs Seafood Kitchen | node     | nan       | nan              |
|  1 | 281474976710657 | ['Airport']    | DEN    | nan                         | node     | Airport   | Denver           |
|  2 | 281474976710656 | ['Airport']    | SJC    | nan                         | node     | Airport   | San Jose         |
|  3 | 562949953421312 | ['Airport']    | MKE    | nan                         | node     | Airport   | Milwaukee        |
|  4 | 562949953421313 | ['Airport']    | ORD    | nan                         | node     | Airport   | Chicago O-Hare   |
+----+----------

          0/? [?op/s]

OperationError: 6j3FMt4qfykfYPvtkCqX4Xo51ZhbJWzVMP4LBCvwSPMk-4QQPiidTRwPbU61MU backtrace (QueryOperation.cpp:67): backtrace (Network.h:331): ANCE (ArrowSerialize.cpp:304): InternalError:ExecutionError: InternalError:ExecutionError: InternalError:ExecutionError

In [ ]:
#  What's in the edge .. 

l_result = my_graph.query("""

   MATCH ( n )  - [r] ->  (m)
   RETURN n.id AS FROM, m.id AS TO, r.DISTANCE AS DISTANCE, r.NUM_HOPS AS NUM_HOPS
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))


In [ ]:
# ##################################################################

In [ ]:

#  Brian's logic thing ..

l_result1 = my_graph.query("""

   MATCH ( n ) 
   RETURN COUNT(*)
   
   """)

display(l_result1)


l_result2 = my_graph.query("""

   MATCH ( n ) 
   WHERE
      (n.id <> 'ORD' OR n.id <> 'MKE')
   
   RETURN COUNT(*)
   
   """)

display(l_result2)


#  Administration/testing stuff ..

In [ ]:
#  Save current graph as an RDG ..

from katana.remote import export_data

l_graph_path = "gs://farrell-bucket/sssp/farrell-sssp"

export_data.rdg(my_graph, l_graph_path)

print("--")


In [ ]:
#  Load from RDG ..
#     Have an empty graph created already ..

from katana.remote import import_data


l_graph_path = "gs://farrell-bucket/sssp/farrell-sssp"

import_data.rdg(my_graph, l_graph_path)

print("--")
